In [1]:
!pip install dalle-pytorch datasets

     |████████████████████████████████| 1.4 MB 4.1 MB/s 
     |████████████████████████████████| 290 kB 51.6 MB/s 
     |████████████████████████████████| 3.1 MB 38.2 MB/s 
     |████████████████████████████████| 47 kB 4.5 MB/s 
     |████████████████████████████████| 51 kB 240 kB/s 
     |████████████████████████████████| 3.3 MB 28.3 MB/s 
     |████████████████████████████████| 1.7 MB 34.4 MB/s 
     |████████████████████████████████| 64 kB 2.4 MB/s 
     |████████████████████████████████| 1.1 MB 38.5 MB/s 
     |████████████████████████████████| 59 kB 7.2 MB/s 
     |████████████████████████████████| 132 kB 33.8 MB/s 
     |████████████████████████████████| 243 kB 49.1 MB/s 
     |████████████████████████████████| 192 kB 48.9 MB/s 
     |████████████████████████████████| 271 kB 51.1 MB/s 
     |████████████████████████████████| 160 kB 49.0 MB/s 
     |████████████████████████████████| 21.5 MB 1.6 MB/s 
     |████████████████████████████████| 65 kB 3.3 MB/s 
     |███████████████████

In [2]:
import numpy as np
import torch
import torchvision.transforms as T
from IPython.display import display
from datasets import load_dataset
from dalle_pytorch.vae import VQGanVAE, download
from tqdm import tqdm

torch.set_grad_enabled(False);

In [ ]:
# VQGAN with downsampling factor f=8, 8192 codebook entries, and Gumbel quantization
# Note: If you change the URLs below, remove ~/.cache/dalle/* to clear the cache
VQGAN_VAE_PATH = 'https://heibox.uni-heidelberg.de/f/34a747d5765840b5a99d/?dl=1'
VQGAN_VAE_CONFIG_PATH = 'https://heibox.uni-heidelberg.de/f/b24d14998a8d4f19a34f/?dl=1'

dev = torch.device('cuda:0')
model = VQGanVAE(vqgan_model_path=download(VQGAN_VAE_PATH, 'vqgan.ckpt'),
                 vqgan_config_path=download(VQGAN_VAE_CONFIG_PATH, 'vqgan_config.yaml')).to(dev)

dataset = load_dataset('laion/laion_100m_vqgan_f8', split='train', streaming=True).shuffle(16384, seed=43)

 92%|█████████████████████  | 345333760/376581823 [00:32<00:02, 11545890.05it/s]

### Decoding embeddings

In [ ]:
import matplotlib.pyplot as plt

for item in dataset.take(5):
    z = torch.tensor(np.frombuffer(item['code'], np.int16)).to(dev, torch.int64)[None]
    x = model.decode(z)
    image = T.ToPILImage(mode='RGB')(x[0].cpu())

    print(f"Caption: {item['caption']}\nURL: {item['url']}")
    display(image)
    print()

### Measuring streaming speed

In [ ]:
n_images = 10 ** 5
for item in tqdm(dataset.take(n_images)):
    pass